In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Load pickled data *Eurosat dataset
import pickle
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [7]:
import tensorflow as tf
# Reshapes raw array data to numpy array for both pixel data and label
X = np.array(X).reshape(-1, 64, 64, 3)
y = np.array(y)

In [8]:
# Image size based on Eurosat data, for RGB Channel
IMG_SIZE = 64
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [9]:
base_model = tf.keras.applications.resnet.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [10]:
base_model.trainable = False
# Let's take a look at the base model architecture
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [11]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [12]:
prediction_layer = tf.keras.layers.Dense(1)

In [13]:
# forming the pretained model from Resnet50 trained on imagenet dataset
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
])

model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.add(tf.keras.layers.Activation('relu'))

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2, 2, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [15]:
len(model.trainable_variables)

2

In [20]:
# Testing with pretained model, now able to obtained above 50% accuracy for unseen data.
# Will continue working on improving the underlying model classification accuracy.
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2)

Train on 21600 samples, validate on 5400 samples
Epoch 1/10
21600/21600 [==============================] - 21s 963us/sample - loss: 0.4403 - accuracy: 0.8612 - val_loss: 1.7196 - val_accuracy: 0.5319
Epoch 2/10
21600/21600 [==============================] - 21s 959us/sample - loss: 0.4046 - accuracy: 0.8713 - val_loss: 1.8418 - val_accuracy: 0.5263
Epoch 3/10
21600/21600 [==============================] - 21s 961us/sample - loss: 0.3900 - accuracy: 0.8756 - val_loss: 1.7689 - val_accuracy: 0.5407
Epoch 4/10
21600/21600 [==============================] - 21s 976us/sample - loss: 0.3715 - accuracy: 0.8802 - val_loss: 1.7990 - val_accuracy: 0.5394
Epoch 5/10
21600/21600 [==============================] - 21s 967us/sample - loss: 0.3556 - accuracy: 0.8849 - val_loss: 1.8822 - val_accuracy: 0.5294
Epoch 6/10
21600/21600 [==============================] - 21s 975us/sample - loss: 0.3445 - accuracy: 0.8878 - val_loss: 1.9077 - val_accuracy: 0.5354
Epoch 7/10
21600/21600 [=====================

In [21]:
# Able to set tf to compute using local gpu
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:01:00.0, compute capability: 6.1

